In [1]:
block_size = 64

final_perm_table = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

init_perm_table = [ final_perm_table.index(i) + 1 for i in range(1, 65) ]

def init_permute(inp):
    output = [None for _ in range(block_size)]
    for char, loc in zip(inp, final_perm_table):
        output[loc - 1] = char
    output = "".join(output)
    return [output[i:i+8] for i in range(0, len(output), 8)]

def final_permute(inp):
    output = [None for _ in range(block_size)]
    for char, loc in zip(inp, init_perm_table):
        output[loc - 1] = char
    output = "".join(output)
    return [output[i:i+8] for i in range(0, len(output), 8)]

def main():
    inp = input('Enter the ascii string to encrypt: ')
    inp = inp.replace(" ", "")
    if len(inp) % 8 is not 0:
        rem = 8 - (len(inp) % 8)
        inp = inp + " " * rem
    
    for i in range(len(inp) // 8):
        block = inp[i*8:(i+1)*8]
        binary = "".join([format(ord(char), '08b') for char in block])
        print("Input Block: ", block, '=', [binary[i:i+8] for i in range(0, len(binary), 8)])
        res_init_perm = init_permute(binary)
        print("After Initial Permutation: ", res_init_perm)
        final_output = final_permute("".join(res_init_perm))
        print("After Final Permutation: ", final_output)
        print('Input Block = Final Permutation Output: ', "".join(final_output) == binary)
        print()

if __name__ == '__main__':
    main()

Enter the ascii string to encrypt: suyash
Input Block:  suyash   = ['01110011', '01110101', '01111001', '01100001', '01110011', '01101000', '00100000', '00100000']
After Initial Permutation:  ['00111111', '00010111', '00000010', '00011111', '00000000', '11111111', '00100100', '00010001']
After Final Permutation:  ['01110011', '01110101', '01111001', '01100001', '01110011', '01101000', '00100000', '00100000']
Input Block = Final Permutation Output:  True



In [7]:
perm_choice_1 = [57, 49, 41, 33, 25, 17, 9,
                1, 58, 50, 42, 34, 26, 18,
                10, 2, 59, 51, 43, 35, 27,
                19, 11, 3, 60, 52, 44, 36,
                63, 55, 47, 39, 31, 23, 15,
                7, 62, 54, 46, 38, 30, 22,
                14, 6, 61, 53, 45, 37, 29,
                21, 13, 5, 28, 20, 12, 4]

perm_choice_2 = [14, 17, 11, 24, 1, 5, 3, 28,
                15, 6, 21, 10, 23, 19, 12, 4,
                26, 8, 16, 7, 27, 20, 13, 2,
                41, 52, 31, 37, 47, 55, 30, 40,
                51, 45, 33, 48, 44, 49, 39, 56,
                34, 53, 46, 42, 50, 36, 29, 32]

bit_shifts = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

def permute(inp, perm_table):
    return [inp[[pos]-1] for pos in perm_table]

def main():
    inp = input('Enter the ascii string key: ')
    inp = inp.replace(" ", "")
    if len(inp) < 8:
        print('Input size less than 7. Need atleast 8 characters')
        return
    
    ch = int(input('Enter 1 or 2 to select bits for key: '))
    if ch == 1:
        inp = inp[:8]
    else:
        inp = inp[-8:]
    
    keys = []
    binary = "".join([format(ord(char), '08b') for char in inp])
    x = permute(binary, perm_choice_1)
    l, r = x[:28], x[28:]

    for i in range(16):
        l, r = l[bit_shifts[i]:] + l[:bit_shifts[i]], r[bit_shifts[i]:] + r[:bit_shifts[i]]
        keys.append("".join(permute(l + r, perm_choice_2)))

    with open('Round-Key.txt', 'w') as f:
        for key in keys:
            print(key)
            f.write("%s\n" % key)

if __name__ == '__main__':
    main()

Enter the ascii string key: thisismykey
Enter 1 or 2 to select bits for key: 2


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [5]:
expansion_table = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

s_box = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
    ],

    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
    ],

    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
    ],

    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
    ],

    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
    ],

    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
    ],

    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12], 
    ],

    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
    ]
]

perm = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25
]

def permute(block, perm_table):
    return [block[x-1] for x in perm_table]

def substitute(exp_r):
    subblocks = [exp_r[k:k+6] for k in range(0, len(exp_r), 6)]
    result = list()
    for i in range(len(subblocks)):
        block = subblocks[i]
        row = int(str(block[0])+str(block[5]),2)
        column = int(''.join([str(x) for x in block[1:][:-1]]),2)
        val = s_box[i][row][column]
        bin = format(val, '04b')
        result += [int(x) for x in bin]
    return result

def main():
    with open('Data.txt', 'r') as f:
        inp = f.read()
        inp = [int(c) for c in inp]
        f.close()
    # print(inp)
    results = []
    with open('Round-Key.txt', 'r') as f:
        keys = f.read().splitlines()
        f.close()

    keys = [[int(c) for c in key.replace(" ", "")] for key in keys]
    # print(keys)

    for i in range(len(inp) // 64):
        block = inp[i*64:(i+1)*64]
        l, r = block[:32], block[32:]
        for round in range(16):
            res = permute(r, expansion_table)
            res = [a^b for a, b in zip(keys[round], res)]
            res = substitute(res)
            res = permute(res, perm)
            res = [a^b for a, b in zip(l, res)]
            l = r
            r = res
        result = "".join([str(b) for b in r+l])
        print(result)
        results.append(result)

    with open('Output-Program-3.txt', 'w') as f:
        f.write("%s\n" % "".join(results))

if __name__ == '__main__':
    main()

ValueError: invalid literal for int() with base 10: 's'